In [1]:
import itertools
import logging
from tqdm import tqdm

from datamaestro import prepare_dataset
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import torch
import torch.nn.functional as F
from typing import List
import time
logging.basicConfig(level=logging.INFO)

ds = prepare_dataset('org.universaldependencies.french.gsd')


# Format de sortie décrit dans
# https://pypi.org/project/conllu/

class Vocabulary:
    """Permet de gérer un vocabulaire.

    En test, il est possible qu'un mot ne soit pas dans le
    vocabulaire : dans ce cas le token "__OOV__" est utilisé.
    Attention : il faut tenir compte de cela lors de l'apprentissage !

    Utilisation:

    - en train, utiliser v.get("blah", adding=True) pour que le mot soit ajouté
      automatiquement
    - en test, utiliser v["blah"] pour récupérer l'ID du mot (ou l'ID de OOV)
    """
    OOVID = 1
    PAD = 0

    def __init__(self, oov: bool):
        self.oov =  oov
        self.id2word = [ "PAD"]
        self.word2id = { "PAD" : Vocabulary.PAD}
        if oov:
            self.word2id["__OOV__"] = Vocabulary.OOVID
            self.id2word.append("__OOV__")

    def __getitem__(self, word: str):
        if self.oov:
            return self.word2id.get(word, Vocabulary.OOVID)
        return self.word2id[word]

    def get(self, word: str, adding=True):
        try:
            return self.word2id[word]
        except KeyError:
            if adding:
                wordid = len(self.id2word)
                self.word2id[word] = wordid
                self.id2word.append(word)
                return wordid
            if self.oov:
                return Vocabulary.OOVID
            raise

    def __len__(self):
        return len(self.id2word)

    def getword(self,idx: int):
        if idx < len(self):
            return self.id2word[idx]
        return None

    def getwords(self,idx: List[int]):
        return [self.getword(i) for i in idx]



class TaggingDataset():
    def __init__(self, data, words: Vocabulary, tags: Vocabulary, adding=True):
        self.sentences = []

        for s in data:
            self.sentences.append(([words.get(token["form"], adding) for token in s], [tags.get(token["upostag"], adding) for token in s]))
    def __len__(self):
        return len(self.sentences)
    def __getitem__(self, ix):
        return self.sentences[ix]


def collate(batch):
    """Collate using pad_sequence"""
    return tuple(pad_sequence([torch.LongTensor(b[j]) for b in batch]) for j in range(2))


logging.info("Loading datasets...")
words = Vocabulary(True)
tags = Vocabulary(False)
train_data = TaggingDataset(ds.train, words, tags, True)
dev_data = TaggingDataset(ds.validation, words, tags, True)
test_data = TaggingDataset(ds.test, words, tags, False)


logging.info("Vocabulary size: %d", len(words))


BATCH_SIZE=100

train_loader = DataLoader(train_data, collate_fn=collate, batch_size=BATCH_SIZE, shuffle=True)
dev_loader = DataLoader(dev_data, collate_fn=collate, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_data, collate_fn=collate, batch_size=BATCH_SIZE)

INFO:root:Loading datasets...
INFO:root:Vocabulary size: 42930


In [3]:
class LSTMTag(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, target_size):
        super(LSTMTag, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, target_size)
        
    def forward(self, sentence):
        
        embeds = self.word_embeddings(sentence)
        lstm_out, (hidden_state, cell_state) = self.lstm(embeds)
        tag_space = self.hidden2tag(lstm_out)
        return tag_space

In [62]:
EMBEDDING_DIM = 32
HIDDEN_DIM = 32


model = LSTMTag(EMBEDDING_DIM, HIDDEN_DIM, len(words), len(tags))
criterion = nn.CrossEntropyLoss(ignore_index=0) # Ignore padding idx : 0
optimizer = optim.Adam(model.parameters(), lr=1e-3)


train_loss, train_accuracy, test_loss, test_accuracy= [], [], [], []
for epoch in range(50):
    # Train

    model.train()
    correct = 0; total = 0
    for sentence, tag in train_loader:
        b,t = target.shape

        # Compute Loss
        target = tag.permute(1,0)
        model.zero_grad()
        tag_scores = model(sentence).permute(1,2,0)
        l_train = criterion(tag_scores, target)
        l_train.backward()
        optimizer.step()

        # Compute accuracy
        predicted = tag_scores.max(1)[1]
        c = (target == predicted) 
        c[torch.where(target[1] == 0)] = False
        correct += c.sum() # nb of correct prediction
        total += b*t - len(torch.where(target == 0)[0]) # nb of tags to predict
        
    train_acc = 100 * correct / float(total)
    train_accuracy.append(train_acc)
    train_loss.append(l_train)

    # Test

    model.eval()
    correct = 0 ; total = 0
    for sentence, tag in dev_loader:
        b,t = target.shape

        # Compute Loss
        target = tag.permute(1,0)
        tag_scores = model(sentence).permute(1,2,0)
        l_test = criterion(tag_scores, target)

        # Compute accuracy
        predicted = tag_scores.max(1)[1]
        c = (target == predicted) 
        c[torch.where(target[1] == 0)] = False
        correct += c.sum() # nb of correct prediction
        total += b*t - len(torch.where(target == 0)[0]) # nb of tags to predict

    test_acc = 100 * correct / float(total)
    test_accuracy.append(test_acc)
    test_loss.append(l_test)


    print(f"epoch {epoch} : Train Loss {'%.2f'%l_train}, Train acc {'%.2f'%train_acc}, Test Loss {'%.2f'%l_test}, Test acc {'%.2f'%test_acc}")

In [85]:
len(torch.where(target == 0)[0])

4153

In [94]:
target[1]

tensor([12, 12, 10,  5,  1, 14,  2, 14,  7, 12,  7, 12, 13,  5,  1, 14,  2, 14,
         7, 12, 12, 12,  7,  1,  3,  2, 10,  4,  1,  2,  7,  2,  3, 10,  9,  5,
         7,  6,  8,  2,  7,  2, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [95]:
predicted[1]

tensor([ 5,  5, 12,  5, 16, 14,  7, 14, 14, 14, 14, 14,  5,  6,  6,  7,  7,  7,
         7,  7, 14, 14, 14, 12,  6,  0,  6, 17, 17, 12, 12, 14, 10, 14, 12, 14,
        14, 14, 14, 14, 14,  7, 17, 12, 12,  7,  7,  7,  7,  7,  7,  7,  7,  7,
         7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7])

In [96]:
b = (target[1] == predicted[1])

In [99]:
b

tensor([False, False, False,  True, False,  True, False,  True, False, False,
        False, False, False, False, False, False, False, False,  True, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False])

In [98]:
b[torch.where(target[1] == 0)] = False

In [ ]:
print("Training Finished!!!\nAgain testing on unknown data")
with torch.no_grad():
    for seq in [seq1, seq2]:
        inputs = prepare_sequence(seq, word_to_ix)
        tag_scores = model(inputs)
        _, indices = torch.max(tag_scores, 1)
        ret = []
        for i in range(len(indices)):
            for key, value in tag_to_ix.items():
                if indices[i] == value:
                    ret.append((seq[i], key))
        print(ret)